In [1]:
import pdb
import os
import json
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as tf_bk
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import (Adam, Nadam, )
from tensorflow.keras.utils import plot_model

from hypermodel_utils import load_args, load_data_chunks

from sklearn.model_selection import StratifiedShuffleSplit

from metrics import margin_loss
import mlflow
from mlflow import keras as mlflow_keras

from promoter_data import PromoterData, DataChunk
from my_generator import AugmentedGeneratorMultipleInputs

import mymodels.parent_models as mymodels
from ml_logs import MlLogs

from promoter_utils import (
    train_test,
    get_model,
    get_test_stats,
)
from hypermodel_utils import (
    load_args,
)

import pprint

# Set seeds
np.random.seed(1337)
if int(str(tf.__version__).split('.')[0]) >= 2:
    from tensorflow import random as tf_random
    tf_random.set_seed(3)
else:
    from tensorflow import set_random_seed
    set_random_seed(3)

Using TensorFlow backend.
C:\Users\lauro\Anaconda3\envs\ml_promoter\lib\site-packages\gensim\corpora\dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, defaultdict
C:\Users\lauro\Anaconda3\envs\ml_promoter\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Load arguments and experiment configurations

config_file_path = 'bacillus_exp_07.yaml'
args_str = '--config {}'.format(config_file_path)
args = load_args(args_str)

Using parameters from: bacillus_exp_07.yaml
{'organism': 'Bacillus', 'model_type': 'HOT_CNN_BACILLUS_02', 'early_stop': 2, 'cv': 2, 'n_samples': 3, 'epochs': 5, 'batch_size': 16, 'fasta_dir': 'fasta', 'weights_dir': 'weights', 'best_weights_dir': 'best_weights', 'base_results_dir': 'results', 'debug': 1, 'data': [{'k': 1, 'encode': 'onehot', 'slice': [59, 59, 20]}]}


In [3]:
# -

for arg in vars(args):
    print(arg, getattr(args, arg))



organism Bacillus
model_type HOT_CNN_BACILLUS_02
cv 2
epochs 5
early_stop 2
n_samples 3
batch_size 16
initial_lr 0.001
fasta_dir fasta
weights_dir weights
best_weights_dir best_weights
base_results_dir results
debug 1
config bacillus_exp_07.yaml
data [{'k': 1, 'encode': 'onehot', 'slice': [59, 59, 20]}]
experiment_name bacillus_exp_07
timestamp 2020_07_16=17_19_30
samples [23, 29, 31]
